In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch import tensor
import numpy as np
import torchvision
from torchvision import datasets,transforms,models
import matplotlib.pyplot as plt
import time
import os
import copy
import torchmetrics
from torchmetrics.classification import Accuracy, Precision, Recall, F1Score

In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [28]:
mean = np.array([0.485,0.456,0.406])
std = np.array([0.229,0.224,0.225])

data_transforms = {
    'train':transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
    'val':transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
}

In [29]:
data_dir = 'train_dataset'
sets = ['train','val']
image_datasets = {x:datasets.ImageFolder(os.path.join(data_dir,x),
                                         data_transforms[x]) 
                  for x in ['train','val']}
dataloaders = {x:torch.utils.data.DataLoader(image_datasets[x],batch_size=4,
                                             shuffle=True,num_workers=0)
                for x in ['train','val']}


In [30]:
dataset_sizes = {x:len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes
print(class_names)

['1', '2', '3']


In [31]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # initialize metric
    metric = torchmetrics.classification.Accuracy(task="multiclass", num_classes=3)
    predicted_labels = []
    ground_truth_labels = []
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-'*10)
        
        #Training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            #Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                #forward
                #track history only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _,preds = torch.max(outputs,1)
                    loss = criterion(outputs,labels)
                    
                    #backward + optimize only in train
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                
                #statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            predicted_labels.append(preds.cpu())
            ground_truth_labels.append(labels.cpu())

            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            #deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
    
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    #calculate accuracy
    predicted_labels = torch.cat(predicted_labels)
    ground_truth_labels = torch.cat(ground_truth_labels)
    accuracy = Accuracy(task="multiclass", num_classes=3)
    accuracy(predicted_labels, ground_truth_labels)
    print(f'Accuracy: {accuracy.compute():.4f}')
    
    #calculate precision
    precision = Precision(task="multiclass", average='macro', num_classes=3)
    precision(predicted_labels, ground_truth_labels)
    print(f'Precision: {precision.compute():.4f}')
    
    #calculate recall
    recall = Recall(task="multiclass", average='macro', num_classes=3)
    recall(predicted_labels, ground_truth_labels)
    print(f'Recall: {recall.compute():.4f}')
    
    #calculate f1 score
    f1 = F1Score(task="multiclass", average='macro', num_classes=3)
    f1(predicted_labels, ground_truth_labels)
    print(f'F1: {f1.compute():.4f}')
    
    #calculate confusion matrix
    cm = torchmetrics.functional.confusion_matrix(predicted_labels, ground_truth_labels, num_classes=3, task="multiclass")
    print(f'Confusion Matrix: \n{cm}')    
    #load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [32]:
pretrained_model = models.vgg16(pretrained=True)

#exchange the last layer
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 3)
pretrained_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pretrained_model.parameters(),lr=0.001)

#scheduler
step_lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)

model_ft = train_model(pretrained_model,criterion,optimizer,step_lr_scheduler,num_epochs=25)

Epoch 0/24
----------


/home/anas/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/anas/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


train Loss: 1.1502 Acc: 0.2889
val Loss: 1.0123 Acc: 0.4667

Epoch 1/24
----------
train Loss: 0.9967 Acc: 0.5333
val Loss: 0.8121 Acc: 0.8000

Epoch 2/24
----------
train Loss: 0.8887 Acc: 0.6000
val Loss: 0.7228 Acc: 0.8667

Epoch 3/24
----------
train Loss: 0.8510 Acc: 0.6000
val Loss: 0.6382 Acc: 0.8667

Epoch 4/24
----------
train Loss: 0.7725 Acc: 0.7556
val Loss: 0.5758 Acc: 0.9333

Epoch 5/24
----------
train Loss: 0.7660 Acc: 0.6667
val Loss: 0.5081 Acc: 0.9333

Epoch 6/24
----------
train Loss: 0.8101 Acc: 0.6889
val Loss: 0.4366 Acc: 1.0000

Epoch 7/24
----------
train Loss: 0.6600 Acc: 0.7778
val Loss: 0.4664 Acc: 1.0000

Epoch 8/24
----------
train Loss: 0.6915 Acc: 0.7556
val Loss: 0.4398 Acc: 1.0000

Epoch 9/24
----------
train Loss: 0.7324 Acc: 0.7556
val Loss: 0.4280 Acc: 1.0000

Epoch 10/24
----------
train Loss: 0.7058 Acc: 0.8000
val Loss: 0.4144 Acc: 1.0000

Epoch 11/24
----------
train Loss: 0.5703 Acc: 0.8444
val Loss: 0.4155 Acc: 1.0000

Epoch 12/24
----------
t

In [33]:
pretrained_model = models.resnet18(pretrained=True)

for param in pretrained_model.parameters():
    param.requires_grad = False
    
#exchange the last layer
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 3)
pretrained_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pretrained_model.parameters(),lr=0.001)

#scheduler
step_lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)

model_ft = train_model(pretrained_model,criterion,optimizer,step_lr_scheduler,num_epochs=25)

Epoch 0/24
----------
train Loss: 1.1424 Acc: 0.4000
val Loss: 1.0305 Acc: 0.4000

Epoch 1/24
----------
train Loss: 1.1012 Acc: 0.4000
val Loss: 1.0350 Acc: 0.3333

Epoch 2/24
----------
train Loss: 1.0612 Acc: 0.4444
val Loss: 0.9528 Acc: 0.6000

Epoch 3/24
----------
train Loss: 1.0133 Acc: 0.4667
val Loss: 0.8885 Acc: 0.6667

Epoch 4/24
----------
train Loss: 0.9009 Acc: 0.6222
val Loss: 0.8523 Acc: 0.7333

Epoch 5/24
----------
train Loss: 0.8667 Acc: 0.6889
val Loss: 0.8046 Acc: 0.6667

Epoch 6/24
----------
train Loss: 0.8729 Acc: 0.6444
val Loss: 0.6754 Acc: 0.9333

Epoch 7/24
----------
train Loss: 0.8294 Acc: 0.7111
val Loss: 0.6870 Acc: 0.9333

Epoch 8/24
----------
train Loss: 0.9360 Acc: 0.6000
val Loss: 0.6868 Acc: 0.8667

Epoch 9/24
----------
train Loss: 0.8655 Acc: 0.6444
val Loss: 0.6565 Acc: 0.9333

Epoch 10/24
----------
train Loss: 0.8863 Acc: 0.6889
val Loss: 0.6965 Acc: 0.9333

Epoch 11/24
----------
train Loss: 0.8933 Acc: 0.6222
val Loss: 0.6802 Acc: 0.9333

Ep